# Grossman HW 6 - Hypothesis Testing

Loading in packages:

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels as sm

os.chdir('/Users/danielgrossman/Documents/Coursework/BMI 312/Data/')
files = os.listdir('/Users/danielgrossman/Documents/Coursework/BMI 312/Data/')

Looping thru each cancer gene and its mutations:

In [ ]:
for f in files:
    if (f.endswith('.somatic.maf.short.gz')):
        print(f, '...')
        
        #read in obs/exp data
        ss = pd.read_csv(f, sep='\t', compression='gzip')
        ge = pd.read_csv('gene_expected.txt', sep='\t')
        
        #create gene name list
        genes = list(set(ss.symbol))
        
        #create mutation type list
        silent, notsilent = [], []
        
        #loop thru each gene and check mutation type counts
        for gene in genes:
            #create list of all mutations per gene
            mutations = list(ss.loc[(ss.symbol == gene), 'var_class'])
            
            #add 'Silent' counts to list
            silent.append(mutations.count('Silent'))
            
            #add notsilent counts to list
            nscounts = len(mutations) - mutations.count('Silent')
            notsilent.append(nscounts)
        
        #combine data into DF
        mut_cnt = pd.DataFrame(list(zip(genes, silent, notsilent)), columns = ['symbol','obs_syn','obs_alt'])            
        
        #adding 'obs_fr' colummn
        mut_cnt['obs_fr'] = mut_cnt.obs_alt/(mut_cnt.obs_syn + mut_cnt.obs_alt)
        
        #descending sort
        des_mut_cnt = mut_cnt.sort_values(['obs_alt'], ascending=False)
        print('Top 10 genes: most protein-altering mutations\n', des_mut_cnt.head(10))
        
        #ascending sort
        asc_mut_cnt = mut_cnt.sort_values(['obs_fr'], ascending=False)
        print('Top 10 genes: highest fraction of protein-altering mutations', asc_mut_cnt.head(10))
        
        #merge 'mut_cnt' with 'gene_expected'
        mut_cnt_more = mut_cnt.merge(ge, on='symbol')
        mut_cnt_more['exp_fr'] = mut_cnt_more.exp_alt/(mut_cnt_more.exp_syn + mut_cnt_more.exp_alt)
        
        #remove genes with obs_fr < exp_fr
        mut_cnt_more = mut_cnt_more.loc[mut_cnt_more.obs_fr >= mut_cnt_more.exp_fr]
        
        #Chi-squared test - using .05 p-value
        ps = []
        for gene in mut_cnt_more:
            p = st.chi2_contingency([mut_cnt_more.obs_syn, mut_cnt_more.obs_alt],[mut_cnt_more.exp_syn, mut_cnt_more.exp_alt])
            ps.append(p)